In [2]:
import pandas as pd
import yfinance as yf

In [3]:
static_data_df = pd.DataFrame(data=[
    ['META', 'USD', 'facebook'],
    ['NVDA', 'USD', 'nvdia'],
    ['FBTC', 'USD', 'bitcoin etf'],
    ['C6L.SI', 'USD', 'singaproe airlines'],
    ['DAL', 'USD', 'Delta airlines'],
    ['EPAM', 'USD', 'Epam systems'],
    ['AAPL', 'USD', 'Apple Inc'],
], columns=['ticker', 'currency', 'desc'])

In [17]:
def find_close(ticker_symbol, precision=2):
    print(ticker_symbol)
    ticker = yf.Ticker(ticker_symbol)
    return round(ticker.history(period='1d')['Close'][-1], precision)

def find_usdsgd_rate():
    return find_close('USDSGD=X', 4)

def find_PnL(price, qty, ticker, side, exchange_rate):
    currency = find_currency(ticker)
    total = price * qty 
    if side == 'B':
        total = total * -1

    if currency=='USD':
        total_in_sgd = total * exchange_rate
    else:
        total_in_sgd = total
    return total_in_sgd
    
def find_total_cost(price, qty):
    return price * qty

def find_currency(ticker):
    return static_data_df[static_data_df['ticker']==ticker]['currency']

In [10]:
transactions = [
    ['20200505', 'META', 8, 208, 'B', 'saxo'],
    ['20200526', 'DAL', 50, 25.2, 'B', 'saxo'],
    ['20200603', 'EPAM', 13, 210, 'B', 'saxo'],
    ['20200603', 'AAPL', 8, 260, 'B', 'saxo'],
    ['20200603', 'AAPL', 5, 250, 'B', 'saxo'],
    ['20200603', 'AAPL', 3, 287, 'B', 'saxo'],
    ['20200608', 'AAPL', 16, 333, 'S', 'saxo'],
    ['20250430', 'META', 7, 531, 'B', 'tiger'],
    ['20250421', 'NVDA', 80, 95.99, 'B', 'tiger'],
    ['20250416', 'FBTC', 54, 73, 'B', 'tiger'],
    ['20250416', 'C6L.SI', 800, 6.30, 'B', 'tiger'],
    ['20250416', 'NVDA', 30, 101, 'B', 'tiger'],
    
]

In [19]:
df = pd.DataFrame(data=transactions, columns=['trade_date', 'ticker', 'qty', 'price', 'side', 'portfolio'])
df['trade_date'] = pd.to_datetime(df['trade_date'])
exchange_rate = find_usdsgd_rate()
df['PnL'] = round(df.apply(lambda row: find_PnL(row['price'], row['qty'], row['ticker'], row['side'], exchange_rate), axis=1), 2)

USDSGD=X


C:\Users\vasan\AppData\Local\Temp\ipykernel_16940\1031458481.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(ticker.history(period='1d')['Close'][-1], precision)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [12]:
df

,trade_date,ticker,qty,price,side,portfolio,purchase_cost
0,2020-05-05,META,8,208.00,B,saxo,1664.0
1,2020-05-26,DAL,50,25.20,B,saxo,1260.0
2,2020-06-03,EPAM,13,210.00,B,saxo,2730.0
3,2020-06-03,AAPL,8,260.00,B,saxo,2080.0
4,2020-06-03,AAPL,5,250.00,B,saxo,1250.0
5,2020-06-03,AAPL,3,287.00,B,saxo,861.0
6,2020-06-08,AAPL,16,333.00,S,saxo,5328.0
7,2025-04-30,META,7,531.00,B,tiger,3717.0
8,2025-04-21,NVDA,80,95.99,B,tiger,7679.2
9,2025-04-16,FBTC,54,73.00,B,tiger,3942.0


In [13]:
agg_df = df.groupby('ticker').agg(
    qty=('qty', 'sum'),
    cost=('purchase_cost', 'sum')
).reset_index()
agg_df['cost_basis'] = agg_df['cost']/agg_df['qty']
agg_df['current_px'] = agg_df.apply(lambda row : find_close(row['ticker'], 2), axis=1)

AAPL
C6L.SI
DAL


C:\Users\vasan\AppData\Local\Temp\ipykernel_16940\520602813.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(ticker.history(period='1d')['Close'][-1], precision)
C:\Users\vasan\AppData\Local\Temp\ipykernel_16940\520602813.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(ticker.history(period='1d')['Close'][-1], precision)
C:\Users\vasan\AppData\Local\Temp\ipykernel_16940\520602813.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by 

EPAM
FBTC
META
NVDA


C:\Users\vasan\AppData\Local\Temp\ipykernel_16940\520602813.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(ticker.history(period='1d')['Close'][-1], precision)
C:\Users\vasan\AppData\Local\Temp\ipykernel_16940\520602813.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return round(ticker.history(period='1d')['Close'][-1], precision)
C:\Users\vasan\AppData\Local\Temp\ipykernel_16940\520602813.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by 

In [14]:
agg_df

,ticker,qty,cost,cost_basis,current_px
0,AAPL,32,9519.0,297.468750,198.53
1,C6L.SI,800,5040.0,6.300000,6.76
2,DAL,50,1260.0,25.200000,48.34
3,EPAM,13,2730.0,210.000000,177.10
4,FBTC,54,3942.0,73.000000,90.12
5,META,15,5381.0,358.733333,592.49
6,NVDA,110,10709.2,97.356364,116.65
